In [1]:
# Instalar spark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.2.3/spark-3.2.3-bin-hadoop3.2.tgz
!tar xf spark-3.2.3-bin-hadoop3.2.tgz
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.3-bin-hadoop3.2"
!pip install -q findspark
!pip install -q pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 21.6 MB/s eta 0:00:00


In [2]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

Los datos adjuntos a esta lección forman parte de la base de datos Football Data from Transfermarkt de Kaggle. El conjunto de datos se compone de varios archivos CSV con información sobre competiciones, juegos, clubes, jugadores y apariciones.

In [3]:
appearances= spark.read.option('header', 'true').option('inferSchema', 'true').csv('./data/appearances.csv')
clubs= spark.read.option('header', 'true').option('inferSchema', 'true').csv('./data/clubs.csv') 
competitions= spark.read.option('header', 'true').option('inferSchema', 'true').csv('./data/competitions.csv')
games= spark.read.option('header', 'true').option('inferSchema', 'true').csv('./data/games.csv')
leagues= spark.read.option('header', 'true').option('inferSchema', 'true').csv('./data/leagues.csv')
players= spark.read.option('header', 'true').option('inferSchema', 'true').csv('./data/players.csv') 

In [4]:
appearances.show()
clubs.show()
competitions.show()
games.show()
leagues.show()
players.show()

+---------+-------+-------------+--------------+--------------+-----+-------+--------------+------------+---------+
|player_id|game_id|appearance_id|competition_id|player_club_id|goals|assists|minutes_played|yellow_cards|red_cards|
+---------+-------+-------------+--------------+--------------+-----+-------+--------------+------------+---------+
|    52453|2483937|2483937_52453|           RU1|         28095|    0|      0|            90|           0|        0|
|    67064|2479929|2479929_67064|           RU1|         28095|    0|      0|            90|           0|        0|
|    67064|2483937|2483937_67064|           RU1|         28095|    0|      0|            90|           0|        0|
|    67064|2484582|2484582_67064|           RU1|         28095|    0|      0|            55|           0|        0|
|    67064|2485965|2485965_67064|           RU1|         28095|    0|      0|            90|           0|        0|
|    67064|2487345|2487345_67064|           RU1|         28095|    0|   

1. Determine los tres países con mayor número de jugadores(jugadores nacidos en ese país). El resultado debe estar ordenado de forma descendente.

In [7]:
from pyspark.sql.functions import desc
players.na.drop(subset=['country_of_birth']).groupBy('country_of_birth').count().orderBy(desc('count')).limit(3).show()

+----------------+-----+
|country_of_birth|count|
+----------------+-----+
|          France| 1694|
|           Spain| 1388|
|           Italy| 1312|
+----------------+-----+



2. Obtenga la lista de jugadores con tarjeta roja. La salida debe contener dos columnas, el nombre de pila del jugador y la cantidad de tarjetas rojas que tiene.

In [14]:
from pyspark.sql.functions import count
appearances.groupBy('player_id').agg(count('red_cards').alias('conteo_rojas')).join(players, 'player_id').select('pretty_name', 'conteo_rojas').orderBy(desc('conteo_rojas')).show()

+-------------------+------------+
|        pretty_name|conteo_rojas|
+-------------------+------------+
|       Ivan Rakitic|         377|
|  Antoine Griezmann|         375|
|       Lionel Messi|         366|
|    Sergio Busquets|         365|
|               Koke|         350|
| Robert Lewandowski|         350|
|      Thomas Muller|         347|
|      Karim Benzema|         344|
|  Cristiano Ronaldo|         343|
|        Dani Parejo|         341|
|        Luis Suarez|         341|
|    Raheem Sterling|         339|
|     Bernardo Silva|         338|
|   Leonardo Bonucci|         337|
|      Jose Callejon|         336|
|        Saul Niguez|         335|
|Georginio Wijnaldum|         334|
|         Toni Kroos|         333|
|      Mohamed Salah|         333|
|    Roberto Firmino|         330|
+-------------------+------------+
only showing top 20 rows



3. ¿Cuántos juegos se jugaron en la Premier League? La salida debe contener dos columnas, el nombre de la liga y la cantidad de juegos que se jugaron en ella.

In [19]:
from pyspark.sql.functions import col
games.groupBy('competition_code').count().join(leagues, col('competition_code') == col('league_id')).select('name', 'count').filter(col('name') == 'premier-league').show()

+--------------+-----+
|          name|count|
+--------------+-----+
|premier-league| 2809|
+--------------+-----+



4. Obtenga las tres ligas con mayor número de asistencia de público teniendo en cuenta todos los juegos que se jugaron en ellas. El resultado debe estar ordenado de forma descendente y tener dos columnas, el nombre de la liga y la asistencia total.

In [25]:
from pyspark.sql.functions import sum
games.groupBy('competition_code').agg(sum('attendance').alias('asistencia_total')).join(leagues, col('competition_code') == col('league_id')).select('name', 'asistencia_total').orderBy(desc('asistencia_total')).show()

+--------------------+----------------+
|                name|asistencia_total|
+--------------------+----------------+
|      premier-league|        86964852|
|          bundesliga|        78102473|
|              laliga|        62943533|
|             serie-a|        53475147|
|             ligue-1|        51593963|
|          eredivisie|        34572418|
|        premier-liga|        20878744|
|            liga-nos|        20072843|
|  jupiler-pro-league|        17817099|
|           super-lig|        17455236|
|scottish-premiership|        17379753|
|         superligaen|         7945555|
|      super-league-1|         6417136|
|        premier-liga|         4944837|
+--------------------+----------------+

